<a href="https://colab.research.google.com/github/tushargupta123/Machine-learning/blob/main/XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.0 MB/s eta 0:00:00


In [2]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/sunnysavita10/svm_kernel/main/Admission_Prediction.csv")

In [4]:
df.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [5]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [6]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [7]:
df["GRE Score"]=df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"]=df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"]=df["University Rating"].fillna(df["University Rating"].median())

In [8]:
X=df.drop(["Serial No.","Chance of Admit"],axis=1)

In [10]:
y=df[['Chance of Admit']]

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=20)

In [12]:
std_sca=StandardScaler()

In [13]:
transformed_x_train = std_sca.fit_transform(X_train)

In [14]:
transformed_x_test = std_sca.transform(X_test)

In [42]:
def objective(trial, data=X, target=y):
    train_x, test_x, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=25)
    param = {
        "tree_method": "gpu_hist",
        "lambda": trial.suggest_float('lambda', 1e-4, 10.0),
        "alpha": trial.suggest_float('alpha', 1e-4, 10.0),
        "colsample_bytree": trial.suggest_categorical('colsample_bytree', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]),
        "subsample": trial.suggest_categorical('subsample', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]),
        "learning_rate": trial.suggest_categorical('learning_rate', [0.00001, 0.0003, 0.008, 0.02, 0.01, 1, 8]),
        "n_estimators": 100,
        "max_depth": trial.suggest_categorical('max_depth', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
        "random_state": trial.suggest_categorical("random_state", [10, 20, 30, 2000, 3454, 22323]),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 200)
    }
    Xgb_reg_model = xgb.XGBRegressor(**param)
    Xgb_reg_model.fit(train_x, y_train,eval_set=[(test_x,y_test)],verbose=True)
    y_pred = Xgb_reg_model.predict(test_x)
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [43]:
find_param = optuna.create_study()
find_param.optimize(objective, n_trials=10)

[I 2023-09-07 10:18:40,934] A new study created in memory with name: no-name-331aeaef-f38a-40cc-a91c-add0f5e6d59a


[0]	validation_0-rmse:0.15151
[1]	validation_0-rmse:0.15064
[2]	validation_0-rmse:0.15064
[3]	validation_0-rmse:0.15110
[4]	validation_0-rmse:0.15110
[5]	validation_0-rmse:0.15110
[6]	validation_0-rmse:0.15110
[7]	validation_0-rmse:0.15110
[8]	validation_0-rmse:0.15062
[9]	validation_0-rmse:0.15077
[10]	validation_0-rmse:0.15077
[11]	validation_0-rmse:0.15077
[12]	validation_0-rmse:0.15077
[13]	validation_0-rmse:0.15178
[14]	validation_0-rmse:0.15066
[15]	validation_0-rmse:0.15051
[16]	validation_0-rmse:0.15223
[17]	validation_0-rmse:0.15060
[18]	validation_0-rmse:0.15060
[19]	validation_0-rmse:0.15060
[20]	validation_0-rmse:0.15095
[21]	validation_0-rmse:0.15095
[22]	validation_0-rmse:0.15095
[23]	validation_0-rmse:0.15143
[24]	validation_0-rmse:0.15143
[25]	validation_0-rmse:0.15066
[26]	validation_0-rmse:0.15069
[27]	validation_0-rmse:0.15091
[28]	validation_0-rmse:0.15084
[29]	validation_0-rmse:0.15084
[30]	validation_0-rmse:0.15084
[31]	validation_0-rmse:0.15059
[32]	validation_0-

[I 2023-09-07 10:18:41,310] Trial 0 finished with value: 0.022728412115780475 and parameters: {'lambda': 6.9458690490042585, 'alpha': 1.3085763823355294, 'colsample_bytree': 0.2, 'subsample': 0.3, 'learning_rate': 1, 'max_depth': 6, 'random_state': 10, 'min_child_weight': 93}. Best is trial 0 with value: 0.022728412115780475.


[0]	validation_0-rmse:0.27220
[1]	validation_0-rmse:0.27219
[2]	validation_0-rmse:0.27219
[3]	validation_0-rmse:0.27219
[4]	validation_0-rmse:0.27219
[5]	validation_0-rmse:0.27219
[6]	validation_0-rmse:0.27218
[7]	validation_0-rmse:0.27218
[8]	validation_0-rmse:0.27218
[9]	validation_0-rmse:0.27218
[10]	validation_0-rmse:0.27218
[11]	validation_0-rmse:0.27217
[12]	validation_0-rmse:0.27217
[13]	validation_0-rmse:0.27217
[14]	validation_0-rmse:0.27217
[15]	validation_0-rmse:0.27217
[16]	validation_0-rmse:0.27216
[17]	validation_0-rmse:0.27216
[18]	validation_0-rmse:0.27216
[19]	validation_0-rmse:0.27216
[20]	validation_0-rmse:0.27215
[21]	validation_0-rmse:0.27215
[22]	validation_0-rmse:0.27215
[23]	validation_0-rmse:0.27215
[24]	validation_0-rmse:0.27215
[25]	validation_0-rmse:0.27214
[26]	validation_0-rmse:0.27214
[27]	validation_0-rmse:0.27214
[28]	validation_0-rmse:0.27214
[29]	validation_0-rmse:0.27214
[30]	validation_0-rmse:0.27213
[31]	validation_0-rmse:0.27213
[32]	validation_0-

[I 2023-09-07 10:18:41,727] Trial 1 finished with value: 0.07397752956758713 and parameters: {'lambda': 1.2143282364347041, 'alpha': 2.8057122927908544, 'colsample_bytree': 0.9, 'subsample': 0.7, 'learning_rate': 1e-05, 'max_depth': 8, 'random_state': 22323, 'min_child_weight': 37}. Best is trial 0 with value: 0.022728412115780475.


...

[I 2023-09-07 10:18:43,906] Trial 7 finished with value: 4.716954245595414e+73 and parameters: {'lambda': 6.497630460563428, 'alpha': 3.0840590851632577, 'colsample_bytree': 0.8, 'subsample': 0.7, 'learning_rate': 8, 'max_depth': 3, 'random_state': 22323, 'min_child_weight': 133}. Best is trial 0 with value: 0.022728412115780475.


[0]	validation_0-rmse:0.27215
[1]	validation_0-rmse:0.27210
[2]	validation_0-rmse:0.27205
[3]	validation_0-rmse:0.27200
[4]	validation_0-rmse:0.27196
[5]	validation_0-rmse:0.27191
[6]	validation_0-rmse:0.27186
[7]	validation_0-rmse:0.27181
[8]	validation_0-rmse:0.27176
[9]	validation_0-rmse:0.27172
[10]	validation_0-rmse:0.27167
[11]	validation_0-rmse:0.27162
[12]	validation_0-rmse:0.27157
[13]	validation_0-rmse:0.27152
[14]	validation_0-rmse:0.27148
[15]	validation_0-rmse:0.27143
[16]	validation_0-rmse:0.27138
[17]	validation_0-rmse:0.27133
[18]	validation_0-rmse:0.27128
[19]	validation_0-rmse:0.27123
[20]	validation_0-rmse:0.27118
[21]	validation_0-rmse:0.27114
[22]	validation_0-rmse:0.27109
[23]	validation_0-rmse:0.27104
[24]	validation_0-rmse:0.27099
[25]	validation_0-rmse:0.27094
[26]	validation_0-rmse:0.27090
[27]	validation_0-rmse:0.27084
[28]	validation_0-rmse:0.27080
[29]	validation_0-rmse:0.27075
[30]	validation_0-rmse:0.27070
[31]	validation_0-rmse:0.27065
[32]	validation_0-

[I 2023-09-07 10:18:44,258] Trial 8 finished with value: 0.07151022200902543 and parameters: {'lambda': 4.337829659892606, 'alpha': 4.423149116522113, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.0003, 'max_depth': 12, 'random_state': 3454, 'min_child_weight': 175}. Best is trial 0 with value: 0.022728412115780475.


[0]	validation_0-rmse:1.63593
[1]	validation_0-rmse:10.52388
[2]	validation_0-rmse:72.75846
[3]	validation_0-rmse:491.30648
[4]	validation_0-rmse:3353.09282
[5]	validation_0-rmse:23176.12398
[6]	validation_0-rmse:155848.23551
[7]	validation_0-rmse:1074470.79730
[8]	validation_0-rmse:7458416.66797
[9]	validation_0-rmse:51774866.37347
[10]	validation_0-rmse:355109228.61404
[11]	validation_0-rmse:2493580695.08164
[12]	validation_0-rmse:16999111498.54744
[13]	validation_0-rmse:118425160727.02254
[14]	validation_0-rmse:820067655907.68274
[15]	validation_0-rmse:5710344444440.77441
[16]	validation_0-rmse:39749336339047.34375
[17]	validation_0-rmse:274338077919608.03125
[18]	validation_0-rmse:1917872942215525.00000
[19]	validation_0-rmse:13385057736609500.00000
[20]	validation_0-rmse:93450099086354560.00000
[21]	validation_0-rmse:646836394296054656.00000
[22]	validation_0-rmse:4453985809969517056.00000
[23]	validation_0-rmse:inf
[24]	validation_0-rmse:inf
[25]	validation_0-rmse:inf
[26]	valida

[I 2023-09-07 10:18:44,684] Trial 9 finished with value: 5.003083537451725e+72 and parameters: {'lambda': 0.8645161777483701, 'alpha': 2.5855854918564805, 'colsample_bytree': 0.7, 'subsample': 0.9, 'learning_rate': 8, 'max_depth': 10, 'random_state': 20, 'min_child_weight': 11}. Best is trial 0 with value: 0.022728412115780475.


In [44]:
find_param.best_trial.params

{'lambda': 6.9458690490042585,
 'alpha': 1.3085763823355294,
 'colsample_bytree': 0.2,
 'subsample': 0.3,
 'learning_rate': 1,
 'max_depth': 6,
 'random_state': 10,
 'min_child_weight': 93}

In [45]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,2.272841e-02,2023-09-07 10:18:40.937402,2023-09-07 10:18:41.309603,0 days 00:00:00.372201,1.308576,0.2,6.945869,1.00000,6,93,10,0.3,COMPLETE
1,1,7.397753e-02,2023-09-07 10:18:41.311406,2023-09-07 10:18:41.726710,0 days 00:00:00.415304,2.805712,0.9,1.214328,0.00001,8,37,22323,0.7,COMPLETE
2,2,7.400017e-02,2023-09-07 10:18:41.730183,2023-09-07 10:18:42.097758,0 days 00:00:00.367575,3.155981,0.9,9.572336,0.00001,11,194,22323,0.7,COMPLETE
3,3,4.802505e-02,2023-09-07 10:18:42.099748,2023-09-07 10:18:42.445660,0 days 00:00:00.345912,7.731097,0.3,1.858181,0.00800,8,37,3454,0.2,COMPLETE
4,4,2.433411e-02,2023-09-07 10:18:42.449052,2023-09-07 10:18:42.810734,0 days 00:00:00.361682,9.083190,0.9,3.312085,1.00000,12,133,10,0.4,COMPLETE
5,5,7.403930e-02,2023-09-07 10:18:42.816103,2023-09-07 10:18:43.159468,0 days 00:00:00.343365,3.666203,0.1,3.721414,0.00001,10,13,2000,0.1,COMPLETE
6,6,3.589219e-02,2023-09-07 10:18:43.161408,2023-09-07 10:18:43.554387,0 days 00:00:00.392979,9.964108,0.4,6.146516,0.01000,11,158,20,0.6,COMPLETE
7,7,4.716954e+73,2023-09-07 10:18:43.556218,2023-09-07 10:18:43.906368,0 days 00:00:00.350150,3.084059,0.8,6.497630,8.00000,3,133,22323,0.7,COMPLETE
8,8,7.151022e-02,2023-09-07 10:18:43.908770,2023-09-07 10:18:44.257580,0 days 00:00:00.348810,4.423149,0.8,4.337830,0.00030,12,175,3454,0.5,COMPLETE
9,9,5.003084e+72,2023-09-07 10:18:44.259915,2023-09-07 10:18:44.684337,0 days 00:00:00.424422,2.585585,0.7,0.864516,8.00000,10,11,20,0.9,COMPLETE


In [46]:
optuna.visualization.plot_optimization_history(find_param)

In [49]:
best_params={'lambda': 6.9458690490042585,
 'alpha': 1.3085763823355294,
 'colsample_bytree': 0.2,
 'subsample': 0.3,
 'learning_rate': 1,
 'max_depth': 6,
 'random_state': 10,
 'min_child_weight': 93}

In [50]:
model=xgb.XGBRegressor(**best_params)

In [51]:
model.fit(X_train,y_train)

XGBRegressor(alpha=1.3085763823355294, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.2, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=6.9458690490042585,
             learning_rate=1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=93, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [52]:
y_pred=model.predict(X_test)

In [53]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

-0.000586213407533176